1. Create an Event Hub instance in the previously created Azure Event Hub namespace.
2. Create a new Shared Access Policy in the Event Hub instance. Copy the connection string generated with the new policy. Note that this connection string has an "EntityPath" component , unlike the RootManageSharedAccessKey connectionstring for the Event Hub namespace.
3. To enable Azure Event hub Databricks ingestion and transformations, install the Azure Event Hubs Connector for Apache Spark from the Maven repository. For this post, I have installed the version 2.3.18 of the connector, using the following  maven coordinate: "com.microsoft.azure:azure-eventhubs-spark_2.12:2.3.18". This library is the most current package at the time of this writing.

In [2]:
# Permission is based on File or folder based ACL assignments to the Data Lake filesystem (container) . RBAC assignments to the top level Azure Data Lake resource is not required.
spark.conf.set("fs.azure.account.auth.type.adlstore.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlstore.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlstore.dfs.core.windows.net", dbutils.secrets.get("myscope", key="clientid"))
spark.conf.set("fs.azure.account.oauth2.client.secret.adlstore.dfs.core.windows.net", dbutils.secrets.get("myscope", key="clientsecret"))
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlstore.dfs.core.windows.net", "https://login.microsoftonline.com/{}/oauth2/token".format(dbutils.secrets.get("myscope", key="tenantid")))

Read stream from Azure Event Hub as streaming dataframe using readStream().
Set your namespace, entity, policy name, and key for Azure Event Hub in the following command.

In [4]:
connectionString = dbutils.secrets.get("myscope", key="eventhubconnstr")

In [5]:
# Initialize event hub config dictionary with connectionString
ehConf = {}
ehConf['eventhubs.connectionString'] = connectionString

In [6]:
# Add consumer group to the ehConf dictionary
ehConf['eventhubs.consumerGroup'] = "$Default"

In [7]:
# Encrypt ehConf connectionString property
ehConf['eventhubs.connectionString'] = sc._jvm.org.apache.spark.eventhubs.EventHubsUtils.encrypt(connectionString)

In [8]:
df = spark.readStream.format("eventhubs").options(**ehConf).load()

In [9]:
display(df)

In [10]:
# Write streams into defined sink
from pyspark.sql.types import *
import  pyspark.sql.functions as F

events_schema = StructType([
  StructField("id", StringType(), True),
  StructField("timestamp", StringType(), True),
  StructField("uv", StringType(), True),
  StructField("temperature", StringType(), True),
  StructField("humidity", StringType(), True)])

decoded_df = df.select(F.from_json(F.col("body").cast("string"), events_schema).alias("Payload"))

In [11]:
display(decoded_df)

In [12]:
df_events = decoded_df.select(decoded_df.Payload.id, decoded_df.Payload.timestamp, decoded_df.Payload.uv, decoded_df.Payload.temperature, decoded_df.Payload.humidity)

In [13]:
display(df_events)

In [14]:
df_out = df_events.writeStream\
  .format("json")\
  .outputMode("append")\
  .option("checkpointLocation", "abfss://checkpointcontainer@adlstore.dfs.core.windows.net/checkpointapievents")\
  .start("abfss://api-eventhub@adlstore.dfs.core.windows.net/writedata")

In [15]:
# read back the data for visual exploration
df_read = spark.read.format("json").load("abfss://api-eventhub@adlstore.dfs.core.windows.net/writedata")

In [16]:
display(df_read)

In [17]:
df_read.printSchema()